<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 95%; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">Hyperparameters Tuning with Optuna</span></b>
  </div>
</div>



<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-danger">

**Contents** 
</div>

<hr>
  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">

  <li> Basic Optuna concepts;  </li>
  <li> Implementation;  </li>
  <li> Visuals;  </li>
  <li> Pruning;  </li>
  <li> Conslusion;  </li>
  <li> Useful Resources  </li>
  
  </ul>
  </font></p>

</div>

<div class="alert alert-warning">

### **Intro** 
</div>

<div class="alert" style="background-color:#E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* In previous lessons, we looked at finding optimal hyperparameters using classical methods: exhaustive grid search (`GridSearchCV`) and random search from a given distribution (`Random Search`).
* Although `Random Search` significantly speeds up the search process, we might miss a set of hyperparameters where the model performs best.<br>
* And here, an idea might come to mind: "What if we do some guessing at first, like in `Random Search`, and then check more often in those areas where the model showed better accuracy?!" This method is called **Bayesian hyperparameter optimization**.
* The most popular libraries implementing this method are `HyperOpt` and `Optuna`. (in our practice, `HyperOpt` often has failures and unstable performance, so in this notebook we will focus on **`Optuna`**)
</div>

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-info">

**Key Features of the `Optuna` Framework**
</div>

* Lightweight and highly versatile - suitable for optimizing arbitrary functions and evaluation metrics.
* Incorporates state-of-the-art algorithms specifically adapted for hyperparameter search.
* Supports parallel execution and advanced pruning strategies.
* Includes built-in tools for result visualization.
* Offers seamless integration with many popular libraries and frameworks (e.g., boosting algorithms, **scikit-learn**, **PyTorch**, **Weights & Biases**, among others).

To understand how to use it effectively, we will examine the framework in detail.

</div>


In [1]:
import optuna

import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, train_test_split

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

**`Optuna` has two core concepts:**


<div class="alert alert-info">

**1. `Study`: an optimization based on an `Objective` function.**
</div>


The `Objective` function should contain the logic for calculating the metric to optimize. Optuna will call this function multiple times to search for the best set of parameters.


<div class="alert" style="background-color:rgb(0, 0, 0); border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color:rgb(255, 255, 255);">

```python
def objective(trial, ...):
    # calculate score...
    return score
```
</div>

<div class="alert alert-info">

**2. `Trial` - a single execution of the `Objective` function**
</div>


Within the `trial` object, we define parameters to be tuned using appropriate methods depending on the type. For example:

<div class="alert" style="background-color:rgb(0, 0, 0); border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color:rgb(255, 255, 255);">

```python
# `suggest_float` method is used to tune float values within the range [0, 1.5]
param = trial.suggest_float('param', 0, 1.5) 

# Categorical value
loss_function = trial.suggest_categorical('loss', ['Logloss', 'CrossEntropy'])

# Integer value
depth = trial.suggest_int('depth', 5, 8)

# Uniform distribution
learning_rate = trial.suggest_uniform('learning_rate', 0.0, 1.0)
```
</div>

</div>

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
[`Optuna`](https://optuna.readthedocs.io/en/stable/index.html) implements several parameter search methods (`samplers`), including classical ones:
* `GridSampler`
* `RandomSampler`
* `Tree-Structured Parzen Estimator` (`TPESampler` – the most popular and default one)
* `BruteForceSampler`
* And [4 more](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html#module-optuna.samplers); you can also implement a custom sampler.
</div>


<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-info">

**Let's go through a simple example to see how `Optuna` works:**
</div>

* Suppose we have a function `y = (x+1)(x+5)(x-9)`, and we want to find the value of `x` that minimizes the function.
* We'll create an `objective` function and pass a `trial` object as an argument — using it, we'll define the search range for `x` as `[-5, 5]`.

<img src='../imgs/05.2.02_1.JPG' width='600px'>

</div>


In [2]:
# Limit logging
optuna.logging.set_verbosity(30)

In [3]:
def objective(trial):
    x = trial.suggest_float("x", -2, 10)
    return (x + 1) * (x + 5) * (x - 9)


# create a study object and run for 10 iterations; since we are searching for a minimum, the direction parameter is left as default
study = optuna.create_study()

# run the optimization
study.optimize(objective,
               n_jobs=-1,
               n_trials=250,
               show_progress_bar=True)

  0%|          | 0/250 [00:00<?, ?it/s]

In [4]:
optuna.visualization.plot_slice(study)

In [5]:
optuna.visualization.plot_optimization_history(study)

In [6]:
# display best parameters
print("Best parameters:", study.best_params)
# display best value
print("Best value:", study.best_value)

Best parameters: {'x': 5.155434824329452}
Best value: -240.32806417898155


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

We see the results of all runs and that the minimum of the function occurred at `x = 5.16`, which is close to the actual minimum.

</div>

In [7]:
from classes import Paths
paths = Paths()
path = paths.quickstart_train
dft = pd.read_csv(path)
dft.head(10)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.10,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.00,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.10,102.382857,2.513319,174,173
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,32.22,engine_fuel,3.880920,1.651883e+07,0.10,172.793237,-5.029476,174,170
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,27.51,engine_fuel,4.181149,1.398317e+07,0.10,203.462289,-14.260456,174,171
5,b12101843B,Skoda Rapid,economy,petrol,2.36,2013,42176,2018,48.99,engine_ignition,4.351782,1.085589e+07,0.10,180.886289,-18.221832,174,173
6,Q-9368117S,Nissan Qashqai,standart,petrol,5.32,2012,24611,2014,54.72,engine_overheat,4.392126,8.343280e+06,0.10,174.984786,12.321364,174,167
7,O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.40,gear_stick,4.712356,9.793288e+06,0.10,95.890736,-8.939366,174,139
8,h16895544p,Kia Sportage,standart,petrol,3.50,2014,56384,2017,33.59,gear_stick,4.507759,1.644405e+07,0.32,101.798615,-1.164690,174,170
9,K77009462l,Smart ForFour,economy,petrol,4.56,2013,41309,2018,39.04,gear_stick,4.376839,6.975742e+06,0.10,125.254983,3.769684,174,173


In [8]:
cat_features = ["model", "car_type", "fuel_type"]  # categorical features
targets = ["target_class", "target_reg"]
features2drop = ["car_id"]  # features to be dropped

In [9]:
# selecting the final set of features for model use
filtered_features = [i for i in dft.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

In [10]:
CatBoostClassifier()

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-info">

**Tips for hyperparameter tuning:**
</div>

- Understand the importance of each parameter
- Set the number of `iterations` with a margin and fix it, while limiting with `early_stopping_rounds`
- Peek at or intuit the value ranges and step sizes
- Exclude parameters that don't need to be tuned (`random_seed`, `eval_metric`, `thread_count`, etc.)
- Use insights from previous runs

</div>


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

Let's define a training function for `CatBoost` that returns predictions using `KFold` validation.

</div>

In [11]:
def fit_catboost(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {
        'iterations' : 400, # No need to tune this parameter, there is Early-Stopping
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 50),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.8),
        
        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 3, 9),
        
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "used_ram_limit": "14gb",
        "eval_metric": "Accuracy", # to be defined beforehand
    }

    
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)
        
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        

    clf = CatBoostClassifier(
        **param,
        thread_count=-1,
        random_seed=42,
        cat_features=cat_features,
    )

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
    )

    y_pred = clf.predict(X_val)
    return clf, y_pred

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

Let's define the `objective` function, in which we will place `KFold` validation to select the best hyperparameters on the entire dataset

</div>

In [12]:
from sklearn.metrics import accuracy_score


def objective(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X_train = dft[filtered_features].drop(targets, axis=1, errors="ignore")
    y_train = dft["target_class"]

    scores, models = [], []
    
    for train_idx, valid_idx in kf.split(X_train):
        train_data = X_train.iloc[train_idx, :], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx, :], y_train.iloc[valid_idx]

        # Pass trials for iteration
        model, y_pred = fit_catboost(trial, train_data, valid_data)  # defined above
        scores.append(accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break
         

    result = np.mean(scores)
    
    if return_models:
        return result, models
    else:
        return result

In [13]:
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=600,
               n_jobs = -1,
               show_progress_bar=True,)

  0%|          | 0/600 [00:00<?, ?it/s]

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

Let's display best parameters

</div>

In [14]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 0.7997432605905006, params {'learning_rate': 0.009139453339059354, 'l2_leaf_reg': 4, 'colsample_bylevel': 0.2441318774346854, 'auto_class_weights': 'Balanced', 'depth': 7, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

Retrain new model on best parameters (selected by Optuna)

</div>

In [15]:
valid_scores, models = objective(
    optuna.trial.FixedTrial(study.best_params),
    return_models=True,
)

In [16]:
valid_scores, len(models)

(0.7997432605905006, 1)

<div class="alert alert-warning">

### **Visualizing Optimization** 
</div>

In [17]:
# in order to display the trials dataframe, we can convert it to a pandas DataFrame
trials_df = study.trials_dataframe().sort_values('value', ascending=False)
trials_df.head(3)

,number,value,datetime_start,datetime_complete,duration,params_auto_class_weights,params_bagging_temperature,params_boosting_type,params_bootstrap_type,params_colsample_bylevel,params_depth,params_l2_leaf_reg,params_learning_rate,params_subsample,state
457,457,0.799743,2025-06-03 23:15:34.925726,2025-06-03 23:15:35.262390,0 days 00:00:00.336664,Balanced,NaN,Plain,MVS,0.244132,7,4,0.009139,NaN,COMPLETE
496,496,0.795892,2025-06-03 23:15:36.684505,2025-06-03 23:15:37.221087,0 days 00:00:00.536582,Balanced,NaN,Plain,MVS,0.367715,8,5,0.009368,NaN,COMPLETE
492,492,0.795892,2025-06-03 23:15:36.403964,2025-06-03 23:15:37.240840,0 days 00:00:00.836876,Balanced,NaN,Plain,MVS,0.317880,8,3,0.009367,NaN,COMPLETE


In [18]:
# Plotting the optimization history by trial number
optuna.visualization.plot_optimization_history(study)

In [19]:
# relative importance of the parameters
params = ['l2_leaf_reg', 'colsample_bylevel', 'bagging_temperature', 'depth', 'bootstrap_type', 'subsample']
optuna.visualization.plot_slice(study,
                                params=params,
                                target_name = 'accuracy_score')

In [20]:
# feature importances
optuna.visualization.plot_param_importances(study)

<div class="alert alert-warning">

### **Pruning** 
</div>

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
`Optuna` implements parallel hyperparameter search — the optimization starts from multiple random points and each evolves along its own unique trajectory.  
`Pruning` — the ability to stop training early when the learning curve worsens compared to previous best results, thereby speeding up the search process.
</div>

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<img src='../imgs/05.2.02_2.png' width='600px'>

</div>


<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
Pruners are implemented for most popular ML frameworks. For example, there are `callbacks` for boosting libraries:

* `CatBoost` : `optuna.integration.CatBoostPruningCallback`
* `XGBoost` : `optuna.integration.XGBoostPruningCallback`
* `LightGBM` : `optuna.integration.LightGBMPruningCallback`
</div>

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
<div class="alert alert-info">

**Main types of pruners:**
</div>

* `Median Pruner` – the most popular one, every few iterations it discards half of the trials with the worst performance
* `Successive Halving Algorithm` (SHA) – starts `trials` with minimal resources (few training examples, few iterations), and at each step discards the worst half while increasing resource allocation
* `Percentile Pruner`, `Hyperband Pruner`, and [others](https://optuna.readthedocs.io/en/stable/reference/pruners.html)
</div>

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
To use pruning in `CatBoost`, let’s slightly modify the `objective` function:
</div>

In [21]:
from optuna.integration import CatBoostPruningCallback


def objective_catboost(trial):
    X = dft[filtered_features].drop(targets, axis=1, errors="ignore")
    y = dft["target_class"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    param = {
        'iterations' : 250,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.01),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 17),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        
        "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
        "depth": trial.suggest_int("depth", 4, 9),
        
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "used_ram_limit": "12gb",
        
        "eval_metric": "Accuracy",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    clf = CatBoostClassifier(
        **param, thread_count=-1, random_seed=42, cat_features=cat_features
    )

    # create Callback instance for pruning
    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")

    clf.fit(
        X_train,
        y_train,
        eval_set=(X_test, y_test),
        verbose=0,
        plot=False,
        early_stopping_rounds=5,
        callbacks=[pruning_callback],
    )  # add the pruning callback to the fit method

    # run the pruning check
    pruning_callback.check_pruned()

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    return accuracy

<div class="alert" style="background-color: #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
    
The pruning function is simply added to the `create_study` function via the `pruner` parameter.  
We will use `MedianPruner`.
</div>

In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    direction="maximize",
)

study.optimize(objective_catboost,
               n_trials=1000,
               n_jobs = -1,
              )

In [24]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 1000
Best trial:
  Value: 0.75
  Params: 
    learning_rate: 0.008393695673503454
    l2_leaf_reg: 7
    colsample_bylevel: 0.09053943122876738
    auto_class_weights: Balanced
    depth: 7
    boosting_type: Ordered
    bootstrap_type: MVS


<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

The search finished noticeably faster, with similar accuracy.
</div>

<div class="alert alert-success">

### **Conclusions and Summary** 
</div>

<div class="alert" style="background-color:rgb(255, 255, 255); border-left: 8px solid #5ad197; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* Hyperparameter tuning usually doesn’t drastically improve model quality — but it does improve it!
* `Optuna` is more efficient than manual/grid/random search.
* `Optuna` offers mechanisms to avoid unnecessary computation.
* Visualization helps eliminate redundant parameters and narrow value ranges.

</div>

<div class="alert alert-success">

### **Useful Resources** 
</div>

<div class="alert" style="background-color:rgb(255, 255, 255); border-left: 8px solid #5ad197; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* [Great article](https://medium.com/@ali.soleymani.co/stop-using-grid-search-or-random-search-for-hyperparameter-tuning-c2468a2ff887) on hyperparameter optimization methods and why `Optuna` is the best best instrument to use in hyperparameter tuning process
* `Optuna` has many more use cases — even [dataset preprocessing](https://t.me/ds_private_sharing/60), limited only by your imagination.
* [Official tutorials](https://optuna.readthedocs.io/en/stable/tutorial/index.html) with examples.
</div>